In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


from tabulate import tabulate

In [5]:
import pathlib
dataset_dir = "drive/MyDrive/Brain_tumor_detection/non_Clahe_dataset/Training"

In [6]:
batch_size = 32
img_height = 256
img_width = 256

In [7]:
from numpy import testing
train_ds = tf.keras.utils.image_dataset_from_directory(
                dataset_dir,
                validation_split=0.2,
                subset="training",
                seed=123,
                image_size=(img_height, img_width),
                batch_size=batch_size)

Found 5712 files belonging to 4 classes.
Using 4570 files for training.


In [8]:
val_ds = tf.keras.utils.image_dataset_from_directory(
                dataset_dir,
                validation_split=0.2,
                subset="validation",
                seed=123,
                image_size=(img_height, img_width),
                batch_size=batch_size)

Found 5712 files belonging to 4 classes.
Using 1142 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)

['glioma', 'meningioma', 'notumor', 'pituitary']


In [10]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 256, 256, 3)
(32,)


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
normalization_layer = layers.Rescaling(1./255)

In [13]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.97647065


In [14]:
from keras.applications.vgg16 import VGG16

In [15]:
vgg16 = VGG16(input_shape=[256,256] + [3], weights='imagenet', include_top=False)


58900480/58889256 [==============================] - 0s 0us/step


In [16]:
for layer in vgg16.layers:
    layer.trainable = False

In [17]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
x = Flatten()(vgg16.output)

In [18]:
prediction = Dense(len(class_names), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
val_normalized_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(val_normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.93529415


In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [22]:
epochs=5
history = model.fit(
    normalized_ds,
    validation_data=val_normalized_ds,
    epochs=epochs
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


143/143 [==============================] - 3688s 26s/step - loss: 0.5018 - accuracy: 0.8282 - val_loss: 0.2527 - val_accuracy: 0.9142
Epoch 2/5
143/143 [==============================] - 3710s 26s/step - loss: 0.1911 - accuracy: 0.9282 - val_loss: 0.2909 - val_accuracy: 0.9011
Epoch 3/5
143/143 [==============================] - 3719s 26s/step - loss: 0.1298 - accuracy: 0.9547 - val_loss: 0.2207 - val_accuracy: 0.9063
Epoch 4/5
143/143 [==============================] - 3732s 26s/step - loss: 0.0778 - accuracy: 0.9740 - val_loss: 0.1943 - val_accuracy: 0.9422
Epoch 5/5
143/143 [==============================] - 3712s 26s/step - loss: 0.0434 - accuracy: 0.9912 - val_loss: 0.1588 - val_accuracy: 0.9457


In [23]:
test_dir = "drive/MyDrive/Brain_tumor_detection/brain_tumor_dataset/Testing"                        

classes = os.listdir("drive/MyDrive/Brain_tumor_detection/brain_tumor_dataset/Testing")
matrix = {
    "actual_Pit_pred_Pit":0,      # not_diseased  tp
    "actual_Pit_pred_no_tumor":0,      # not_diseased  tp
    "actual_Pit_pred_men":0,      # not_diseased  tp
    "actual_Pit_pred_gli":0,      # not_diseased  tp
    "actual_no_tumor_pred_no_tumor":0,      # not_diseased  tp
    "actual_no_tumor_pred_Pit":0,      # not_diseased  tp
    "actual_no_tumor_pred_men":0,      # not_diseased  tp
    "actual_no_tumor_pred_gli":0,      # not_diseased  tp
    "actual_men_pred_men":0,      # not_diseased  tp
    "actual_men_pred_Pit":0,      # not_diseased  tp
    "actual_men_pred_no_tumor":0,      # not_diseased  tp
    "actual_men_pred_gli":0,      # not_diseased  tp
    "actual_gli_pred_gli":0,      # not_diseased  tp
    "actual_gli_pred_Pit":0,      # not_diseased  tp
    "actual_gli_pred_no_tumor":0,      # not_diseased  tp
    "actual_gli_pred_men":0      # not_diseased  tp
    
}
i, s = 0, 0
for classe in classes[::]:
    i+=1
    path = f"{test_dir}/{classe}/"
    images = os.listdir(path)[::]
    for image in images:
        img = tf.keras.utils.load_img(path+image, 
                                    target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        actual, predicted = classe, class_names[np.argmax(score)]
        if actual == "pituitary" and predicted == "pituitary":
            matrix["actual_Pit_pred_Pit"] += 1

        elif actual == "pituitary" and predicted == "notumor":
            matrix["actual_Pit_pred_no_tumor"] += 1

        elif actual == "pituitary" and predicted == "meningioma":
            matrix["actual_Pit_pred_men"] += 1

        elif actual == "pituitary" and predicted == "glioma":
            matrix["actual_Pit_pred_gli"] += 1

        elif actual == "notumor" and predicted == "notumor":
            matrix["actual_no_tumor_pred_no_tumor"] += 1

        elif actual == "notumor" and predicted == "pituitary":
            matrix["actual_no_tumor_pred_Pit"] += 1

        elif actual == "notumor" and predicted == "meningioma":
            matrix["actual_no_tumor_pred_men"] += 1

        elif actual == "notumor" and predicted == "glioma":
            matrix["actual_no_tumor_pred_gli"] += 1

        elif actual == "meningioma" and predicted == "meningioma":
            matrix["actual_men_pred_men"] += 1

        elif actual == "meningioma" and predicted == "pituitary":
            matrix["actual_men_pred_Pit"] += 1

        elif actual == "meningioma" and predicted == "notumor":
            matrix["actual_men_pred_no_tumor"] += 1

        elif actual == "meningioma" and predicted == "glioma":
            matrix["actual_men_pred_gli"] += 1

        elif actual == "glioma" and predicted == "glioma":
            matrix["actual_gli_pred_gli"] += 1

        elif actual == "glioma" and predicted == "pituitary":
            matrix["actual_gli_pred_Pit"] += 1

        elif actual == "glioma" and predicted == "notumor":
            matrix["actual_gli_pred_no_tumor"] += 1

        elif actual == "glioma" and predicted == "meningioma":
            matrix["actual_gli_pred_men"] += 1
     

        if s%100 == 0:
            print(f"{round(s*100/len(os.listdir(test_dir)))}%")
        s +=1
        
        print(
           " prediction: {} and ground truth {} : {:.2f} ."
           .format(class_names[np.argmax(score)], classe, 100 * np.max(score)))


0%
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: meningioma and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: meningioma and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: meningioma and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: notumor and ground truth pituitary : 47.54 .
 prediction: meningioma and ground truth pituitary : 47.54 .

In [24]:
matrix

{'actual_Pit_pred_Pit': 4,
 'actual_Pit_pred_gli': 0,
 'actual_Pit_pred_men': 74,
 'actual_Pit_pred_no_tumor': 222,
 'actual_gli_pred_Pit': 0,
 'actual_gli_pred_gli': 22,
 'actual_gli_pred_men': 88,
 'actual_gli_pred_no_tumor': 190,
 'actual_men_pred_Pit': 0,
 'actual_men_pred_gli': 0,
 'actual_men_pred_men': 167,
 'actual_men_pred_no_tumor': 139,
 'actual_no_tumor_pred_Pit': 0,
 'actual_no_tumor_pred_gli': 0,
 'actual_no_tumor_pred_men': 0,
 'actual_no_tumor_pred_no_tumor': 405}